# Analysis - Python portion

In [1]:
from collections import OrderedDict

import pandas as pd

## Miami-Dade

The gaps were often wide: some schools in Miami-Dade County dropped to around a 75 percent participation rate for remote learning in April, while many of their peers neared full, 100 percent participation.

In [2]:
miami = pd.read_csv('../data/input/foia/miami-dade.csv')
miami.sort_values(by='pct_present').head(10)

,school_number,school_name,region,pct_present,date,distance
327,8151,ROBERT RENICK ED CENTER,NORTH,51.7,4-6-2020,True
119,3181,MELROSE ELEMENTARY,CENTRAL,51.8,4-6-2020,True
449,3181,MELROSE ELEMENTARY,CENTRAL,54.2,4-7-2020,True
779,3181,MELROSE ELEMENTARY,CENTRAL,54.5,4-8-2020,True
163,4501,POINCIANA PARK ELEMENTARY,CENTRAL,57.2,4-6-2020,True
1109,3181,MELROSE ELEMENTARY,CENTRAL,57.9,4-9-2020,True
493,4501,POINCIANA PARK ELEMENTARY,CENTRAL,57.9,4-7-2020,True
2759,3181,MELROSE ELEMENTARY,CENTRAL,58.2,4-13-2020,True
109,2981,LIBERTY CITY ELEMENTARY,CENTRAL,59.2,4-6-2020,True
215,5971,NATHAN B YOUNG ELEMENTARY,NORTH,59.4,4-6-2020,True


In [3]:
miami[miami.date == '4-6-2020'].sort_values(by='pct_present').tail()

,school_number,school_name,region,pct_present,date,distance
300,7291,MAST @ JOSE MARTI 6-12 ACAD,NORTH,97.7,4-6-2020,True
283,7081,DESIGN & ARCHITECTURE SR,CENTRAL,97.8,4-6-2020,True
150,4221,PALMETTO ELEMENTARY,SOUTH,98.1,4-6-2020,True
279,7055,YOUNG WOMENS PREP ACADEMY,CENTRAL,98.3,4-6-2020,True
96,2641,KENDALE ELEMENTARY,SOUTH,99.2,4-6-2020,True


In [4]:
print(len(miami))
print(len(miami[miami.pct_present > 95]))

5940
1810


In [5]:
characteristics = pd.read_csv('../data/intermediary/scorecard/characteristics/miami-dade.csv')
equity = pd.read_csv('../data/intermediary/scorecard/equity/miami-dade.csv')

In [6]:
df = miami.merge(characteristics, on='school_number')
df = df.merge(equity,on='school_number')
df[['school_name', 'date', 'pct_present', 'poverty', 'minority', 'grade']].to_csv('../data/output/miami-dade.csv')

## Broward

In [7]:
broward = pd.read_csv('../data/output/broward.csv')
broward[broward.school_name.str.contains('MCARTHUR') | broward.school_name.str.contains('CYPRESS BAY')]

,Unnamed: 0,school_name,school_number,covid.agg.enroll,covid.agg.attend,covid.avg,district_number,district_name,1819.agg.attend,1819.agg.enroll,1819.avg,minority,poverty,title_i,school_type,grade
41,41,CYPRESS BAY HIGH,3623,52135,49737,95.400403,6,BROWARD,825284,862787,95.653272,Mid-Range Minority,Low Poverty,Non-Title I,High School,A
107,107,MCARTHUR HIGH,241,21339,18815,88.171892,6,BROWARD,334725,364973,91.712264,High Minority,Mid-Range Poverty,Non-Title I,High School,B


### Alternative Schools

In [8]:
broward = pd.read_csv('../data/input/foia/broward.csv')
characteristics = pd.read_csv('../data/intermediary/scorecard/characteristics/broward.csv')
equity = pd.read_csv('../data/intermediary/scorecard/equity/broward.csv')
keys = pd.read_csv('../data/intermediary/keys/broward.csv')

In [9]:
broward = broward[['SchoolName', "Enrollment", "Date"]]
broward['Date'] = broward['Date'].astype('datetime64')
broward.set_index(['Date'],inplace=True)

In [10]:
rows = []
for _, d in broward.groupby('SchoolName', sort=False):
    SchoolName = d['SchoolName'][0]
    EnrollmentStart = d['Enrollment'][0]
    EnrollmentEnd = d['Enrollment'][-1]
    row = OrderedDict([
        ('SchoolName' , SchoolName),
        ('EnrollmentStart' , EnrollmentStart),
        ('EnrollmentEnd' , EnrollmentEnd),
        ('EnrollmentDifference', EnrollmentEnd - EnrollmentStart),
        ('EnrollmentChange', (EnrollmentEnd - EnrollmentStart) / EnrollmentStart * 100)
    ])
    rows.append(row)

enrollment = pd.DataFrame(rows)
enrollment = enrollment.rename(columns={'SchoolName':'school_name'})

In [11]:
keys = keys[['school_name', 'school_number']]

analysis = enrollment.merge(keys, on='school_name')
analysis = analysis.merge(equity, on='school_number')
analysis = analysis.merge(characteristics, on='school_number')

In [12]:
# largest decrease in enrollment
analysis.sort_values(by="EnrollmentChange").head(5)

,school_name,EnrollmentStart,EnrollmentEnd,EnrollmentDifference,EnrollmentChange,school_number,district_number_x,minority,poverty,title_i,district_number_y,school_type,grade,charter,alt_schl,federal_index,CEP_Percentage
45,DAVE THOMAS EDUCATION CENTER WEST,551,509,-42,-7.622505,3651,6,High Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,11.0,75.8
79,HENRY D. PERRY EDUCATION CENTER,1678,1579,-99,-5.899881,6501,6,Mid-Range Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,11.0,20.7
173,SEAGULL SCHOOL,273,269,-4,-1.465201,601,6,High Minority,Mid-Range Poverty,Non-Title I,6,High School,Improvement Plan,NO,A,9.0,80.4
212,WHIDDON ROGERS EDUCATION CENTER,767,759,-8,-1.043025,452,6,High Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,19.0,68.6
42,CYPRESS ELEMENTARY,689,683,-6,-0.870827,1781,6,High Minority,High Poverty,Title I,6,Elementary School,B,NO,N,56.0,90.0


In [13]:
# Only alternative schools
analysis[analysis['alt_schl'] != 'N'].sort_values(by="EnrollmentChange")

,school_name,EnrollmentStart,EnrollmentEnd,EnrollmentDifference,EnrollmentChange,school_number,district_number_x,minority,poverty,title_i,district_number_y,school_type,grade,charter,alt_schl,federal_index,CEP_Percentage
45,DAVE THOMAS EDUCATION CENTER WEST,551,509,-42,-7.622505,3651,6,High Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,11.0,75.8
79,HENRY D. PERRY EDUCATION CENTER,1678,1579,-99,-5.899881,6501,6,Mid-Range Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,11.0,20.7
173,SEAGULL SCHOOL,273,269,-4,-1.465201,601,6,High Minority,Mid-Range Poverty,Non-Title I,6,High School,Improvement Plan,NO,A,9.0,80.4
212,WHIDDON ROGERS EDUCATION CENTER,767,759,-8,-1.043025,452,6,High Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,19.0,68.6
195,THE QUEST CENTER,122,122,0,0.000000,1021,6,Mid-Range Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,E,11.0,62.6
93,LANIER-JAMES EDUCATION CENTER,126,128,2,1.587302,405,6,High Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,0.0,75.9
39,CROSS CREEK SCHOOL,152,155,3,1.973684,3222,6,Mid-Range Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,E,34.0,81.9
213,WHISPERING PINES EXCEPTIONAL EDUCATION CENTER,133,137,4,3.007519,1752,6,Mid-Range Minority,Mid-Range Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,23.0,75.7
43,CYPRESS RUN ALTERNATIVE CENTER,158,165,7,4.430380,2123,6,High Minority,High Poverty,Non-Title I,6,Combination School,Improvement Plan,NO,A,0.0,88.2


## Hillsborough

In [14]:
hillsborough = pd.read_csv('../data/input/foia/hillsborough.csv')
hillsborough['PRESENT'] = hillsborough['ENROLLMENT'] - hillsborough['ABSENT']
hillsborough.sort_values(by="% PRESENT").head(6)

,SCHOOLS,ENROLLMENT,ABSENT,% ABSENT,% PRESENT,SCHOOL_TYPE,WEEK,PRESENT
0,WEST UNIVERSITY CHARTER H.S.,278,166,0.5971,0.4029,CHARTER,4/3/2020,112
1,SEMINOLE HEIGHTS CHARTER SCH,239,135,0.5649,0.4351,CHARTER,4/3/2020,104
296,WEST UNIVERSITY CHARTER H.S.,279,146,0.5233,0.4767,CHARTER,4/10/2020,133
592,SPORTS LEADERSHIP & MANAG ACAD,411,193,0.4696,0.5304,CHARTER,4/17/2020,218
297,SEMINOLE HEIGHTS CHARTER SCH,226,106,0.4690,0.5310,CHARTER,4/10/2020,120
298,GAITHER HIGH SCHOOL,2052,922,0.4493,0.5507,HIGH,4/10/2020,1130


In [15]:
len(hillsborough[hillsborough.ABSENT == 0])

356

In [16]:
hillsborough[
    (hillsborough['SCHOOL_TYPE'] == 'HIGH') &
    (hillsborough['WEEK'] == "4/10/2020")
].sort_values(by="% PRESENT", ascending=False).head()

,SCHOOLS,ENROLLMENT,ABSENT,% ABSENT,% PRESENT,SCHOOL_TYPE,WEEK,PRESENT
580,SICKLES HIGH SCHOOL,2414,0,0.0000,1.0000,HIGH,4/10/2020,2414
572,NEWSOME HIGH SCHOOL,2995,0,0.0000,1.0000,HIGH,4/10/2020,2995
563,LETO HIGH SCHOOL,2027,0,0.0000,1.0000,HIGH,4/10/2020,2027
535,ARMWOOD HIGH SCHOOL,2168,0,0.0000,1.0000,HIGH,4/10/2020,2168
524,FREEDOM HIGH SCHOOL,1746,4,0.0023,0.9977,HIGH,4/10/2020,1742


In [17]:
characteristics[characteristics['school_number'] == 241]

,district_number,school_number,school_type,grade,charter,alt_schl,federal_index,CEP_Percentage
155,6,241,High School,B,NO,N,54.0,71.1


In [18]:
equity[equity['school_number'] == 241]

,school_number,district_number,minority,poverty,title_i
17,241,6,High Minority,Mid-Range Poverty,Non-Title I


## Overall

In [19]:
hillsborough = pd.read_csv('../data/input/foia/hillsborough.csv')
broward = pd.read_csv('../data/input/foia/broward.csv')
miami = pd.read_csv('../data/input/foia/miami-dade.csv')

total_schools = hillsborough.SCHOOLS.nunique() + broward.SchoolName.nunique() + miami.school_name.nunique()
total_schools

845